In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/doc_embedding/doc2vec_5.csv.gz', compression='gzip', low_memory=False, lineterminator='\n', index_col=None)
data

,Unnamed: 0,id,text,create_at,geo,retweet_count,like_count,hashtags,username,following,followers,user_total_tweets,user_likes_count,cleaned_text,0,1,2,3,4
0,0,1.591219e+18,'Like a 1930s Dictator': Vitalik Buterin Lays ...,2022-11-11 23:59:59+00:00,"For now, Earth",0.0,1.0,#ftx,johnmorganFL,630.0,28492.0,240872.0,111.0,like dictator vitalik buterin lay sam bankmanf...,-0.190527,-0.363088,-0.311832,0.111005,-0.058337
1,1,1.591219e+18,"💸 $1,000 #Giveaway 💸\n\nWe'll select 10 winner...",2022-11-11 23:59:57+00:00,Panama,8866.0,0.0,#ftx,KuKzNFT,594.0,1014.0,4452.0,8272.0,giveaway select winner randomly award account...,-0.037449,-0.546747,-0.165052,-0.033582,0.046071
2,2,1.591219e+18,#FTX crazy Shit! #Crypto https://t.co/2GO4X3u2pt,2022-11-11 23:59:55+00:00,"Seattle, WA",0.0,0.0,#ftx,MeAndMySports,315.0,342.0,3032.0,1572.0,ftx crazy shit crypto,0.025254,-0.219788,-0.274583,0.085095,-0.137927
3,3,1.591219e+18,"Yo @SBF_FTX, draw me like one of your French g...",2022-11-11 23:59:53+00:00,"London, England",0.0,0.0,#ftx,thedoc7er,1513.0,296.0,4541.0,3626.0,draw like french girl ftxcrash ftx binance czb...,-0.170994,-0.371573,-0.023886,-0.041357,-0.136645
4,4,1.591219e+18,@BitcoinMagazine you you simpletons cant under...,2022-11-11 23:59:51+00:00,NaN,0.0,1.0,#ftx,jesseypaul4,123.0,44.0,317.0,81.0,simpleton understand crypto wallet like exodus...,-0.198903,-0.373405,-0.227012,0.158047,-0.044631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157503,157503,1.591565e+18,"🔴 WAR IN 🇺🇦: FRANCE 24 reporter @cntrentF24, w...",2022-11-12 22:52:11+00:00,Hauts de France 🇫🇷 🇪🇺,20.0,0.0,#ukraine,annetteashley61,10522.0,9603.0,437088.0,291392.0,war france reporter spend lot time report ukra...,-0.062705,-0.128675,-0.055065,0.015298,-0.156906
157504,157504,1.591565e+18,People still celebrating in Kherson and I love...,2022-11-12 22:52:09+00:00,"Belgrade, Serbia, Europe",1041.0,0.0,#ukraine,svrhovac,222.0,150.0,9174.0,16675.0,people celebrate kherson love bit ukraine kherson,-0.058628,-0.215833,-0.425018,0.128180,-0.058321
157505,157505,1.591565e+18,A remarkable story #Ukraine https://t.co/eeBLc...,2022-11-12 22:52:04+00:00,Guernsey,0.0,0.0,#ukraine,TimBarker10,2668.0,1217.0,3290.0,5927.0,remarkable story ukraine,0.029643,-0.241812,-0.006721,0.150215,-0.148847
157506,157506,1.591565e+18,#Ukraine: Relief in liberated #Kherson after 8...,2022-11-12 22:52:02+00:00,🇮🇪🇪🇺🇺🇦🎮,189.0,0.0,#ukraine,Aontaithe2021,5001.0,2376.0,392315.0,308109.0,ukraine relief liberated kherson month putin ...,0.223750,-0.170265,0.191466,0.264151,-0.158802


In [3]:
data.columns

Index(['Unnamed: 0', 'id', 'text', 'create_at', 'geo', 'retweet_count',
       'like_count', 'hashtags', 'username', 'following', 'followers',
       'user_total_tweets', 'user_likes_count', 'cleaned_text', '0', '1', '2',
       '3', '4'],
      dtype='object')

In [4]:
data['hashtags'].value_counts()

#ukraine          27000
#covid            27000
#christmas        26692
#vegan            26651
#climatechange    26089
#ftx              24076
Name: hashtags, dtype: int64

In [5]:
data['like_count_log'] = np.log(data['like_count']+1)
data['retweet_count_log'] = np.log(data['retweet_count']+1)

get useful columns

In [6]:
useful_columns = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count',
    'retweet_count_log', 
    'like_count_log',
    '0','1','2','3','4'
]

all_features = [
    'following',
    'followers', 
    'user_total_tweets',
    'user_likes_count', 
    '0','1','2','3','4'
]
data_baseline = data[useful_columns]

In [7]:
data_baseline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157508 entries, 0 to 157507
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   following          157508 non-null  float64
 1   followers          157508 non-null  float64
 2   user_total_tweets  157508 non-null  float64
 3   user_likes_count   157508 non-null  float64
 4   retweet_count_log  157508 non-null  float64
 5   like_count_log     157508 non-null  float64
 6   0                  157508 non-null  float64
 7   1                  157508 non-null  float64
 8   2                  157508 non-null  float64
 9   3                  157508 non-null  float64
 10  4                  157508 non-null  float64
dtypes: float64(11)
memory usage: 13.2 MB


baseline (dummy)

In [8]:
features = data_baseline[all_features]
features.head(1)

,following,followers,user_total_tweets,user_likes_count,0,1,2,3,4
0,630.0,28492.0,240872.0,111.0,-0.190527,-0.363088,-0.311832,0.111005,-0.058337


In [9]:
labels_like = data_baseline['like_count_log']
labels_like.describe()

count    157508.000000
mean          0.215542
std           0.648805
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           8.841159
Name: like_count_log, dtype: float64

In [10]:
labels_retweet = data_baseline['retweet_count_log']
labels_retweet.describe()

count    157508.000000
mean          3.121754
std           2.623077
min           0.000000
25%           0.693147
50%           2.708050
75%           5.298317
max          11.903974
Name: retweet_count_log, dtype: float64

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_like, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

dummy1 = DummyRegressor(strategy='mean')
dummy1.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dummy1.predict(X_test_scaled))

0.4286044816224323

In [14]:
dummy2 = DummyRegressor(strategy='constant', constant=0)
dummy2.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dummy2.predict(X_test_scaled))

0.47470842451784023

simple models (selected features)

In [15]:
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)
mean_squared_error(y_test, lr.predict(X_test_scaled))

0.42064415257864357

In [16]:
dt = DecisionTreeRegressor()
dt.fit(X_train_scaled,y_train)
mean_squared_error(y_test, dt.predict(X_test_scaled))

0.6015180273662852

In [17]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train_scaled,y_train)
mean_squared_error(y_test, gbr.predict(X_test_scaled))

0.3538346351626301

In [18]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train_scaled,y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled))

0.30782779386281156

In [19]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
mean_squared_error(y_test, knn.predict(X_test_scaled))

0.49652487114110955

In [21]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_retweet, test_size=0.2, random_state=42)

In [22]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
dummy1 = DummyRegressor(strategy='mean')
dummy1.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dummy1.predict(X_test_scaled))

6.894205733434873

In [24]:
dummy2 = DummyRegressor(strategy='constant', constant=0)
dummy2.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dummy2.predict(X_test_scaled))

16.62982154196123

In [25]:
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)
mean_squared_error(y_test, lr.predict(X_test_scaled))

6.676356539783323

In [26]:
dt = DecisionTreeRegressor()
dt.fit(X_train_scaled,y_train)
mean_squared_error(y_test, dt.predict(X_test_scaled))

8.770917134935162

In [27]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train_scaled,y_train)
mean_squared_error(y_test, gbr.predict(X_test_scaled))

5.368004907041866

In [28]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train_scaled,y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled))

4.453613629984794

In [29]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
mean_squared_error(y_test, knn.predict(X_test_scaled))

7.797391784452629